### Import necessary Library

In [1]:
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

import parselmouth
from parselmouth.praat import call
from pathlib import Path

from metrics import AudioMetrics
from metrics import AudioMetrics2
#from Audio_metrics import AudioMetrics2
import noise_addition_utils
from pypesq import pesq
import shutil


import io
import os
import sys
import math
#import tarfile
import multiprocessing

import scipy
import librosa
import soundfile as sf

import requests
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from IPython.display import Audio, display

print(torch.__version__)
print(torchaudio.__version__)

D:\Master\Thesis\FRCRN\Summary\metrics_utils.py:50: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def lpcoeff(speech_frame, model_order):
D:\Master\Thesis\FRCRN\Summary\noise_addition_utils.py:15: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


2.1.0+cu118
2.1.0+cu118


In [2]:
#-------------------------------------------------------------------------------
# Helper functions.
#-------------------------------------------------------------------------------

def denoise_audio(input_path, output_path, smoothing_factor):
    # Load the audio file
    snd = parselmouth.Sound(input_path)

    # Apply smoothing to reduce noise
    snd_denoised = snd.copy()
    #snd_denoised = call(snd_denoised, "Smooth", smoothing_factor)
    snd_denoised = call(snd_denoised, "Reduce noise", 0.0, 0.0, 0.025, 80.0, 10000.0, 40.0,-20, "Spectral-subtraction")
    # Save the denoised audio
    snd_denoised.save(output_path, "WAV")
def get_stats(waveform, sample_rate=None, src=None):
    max_ = waveform.max().numpy()
    min_ = waveform.min().numpy()
    mean_ = waveform.mean().numpy()
    std_ = waveform.std().numpy()
    return max_,min_,mean_,std_

def print_stats(waveform, sample_rate=None, src=None):
    if src:
        print("-" * 10)
        print("Source:", src)
        print("-" * 10)
    if sample_rate:
        print("Sample Rate:", sample_rate)
        print("Shape:", tuple(waveform.shape))
        print("Dtype:", waveform.dtype)
        print(f" - Max:     {waveform.max().item():6.3f}")
        print(f" - Min:     {waveform.min().item():6.3f}")
        print(f" - Mean:    {waveform.mean().item():6.3f}")
        print(f" - Std Dev: {waveform.std().item():6.3f}")
        print()
        print(waveform)
        print()
    # max_ = waveform.max().numpy()
    # min_ = waveform.min().numpy()
    # mean_ = waveform.mean().numpy()
    # std_ = waveform.std().numpy()
    # return max_,min_,mean_,std_

def plot_waveform(waveform, sample_rate, title="Waveform", xlim=None, ylim=None):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  time_axis = torch.arange(0, num_frames) / sample_rate

  figure, axes = plt.subplots(num_channels, 1)
  if num_channels == 1:
    axes = [axes]
  for c in range(num_channels):
    axes[c].plot(time_axis, waveform[c], linewidth=1)
    axes[c].grid(True)
    if num_channels > 1:
      axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
      axes[c].set_xlim(xlim)
    if ylim:
      axes[c].set_ylim(ylim)
  figure.suptitle(title)
  plt.show(block=False)

def plot_specgram(waveform, sample_rate, title="Spectrogram", xlim=None):
  waveform = waveform.numpy()
  num_channels, num_frames = waveform.shape
  time_axis = torch.arange(0, num_frames) / sample_rate
  figure, axes = plt.subplots(num_channels, 1)
  if num_channels == 1:
    axes = [axes]
  for c in range(num_channels):
    axes[c].specgram(waveform[c], Fs=sample_rate)
    if num_channels > 1:
      axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
      axes[c].set_xlim(xlim)
  figure.suptitle(title)
  plt.show(block=False)

def play_audio(waveform, sample_rate):
  waveform = waveform.numpy()
  num_channels, num_frames = waveform.shape
  if num_channels == 1:
    display(Audio(waveform[0], rate=sample_rate))
  elif num_channels == 2:
    display(Audio((waveform[0], waveform[1]), rate=sample_rate))
  else:
    raise ValueError("Waveform with more than 2 channels are not supported.")

def plot_spectrogram(spec, title=None, ylabel='freq_bin', aspect='auto', xmax=None):
  fig, axs = plt.subplots(1, 1)
  axs.set_title(title or 'Spectrogram (db)')
  axs.set_ylabel(ylabel)
  axs.set_xlabel('frame')
  im = axs.imshow(librosa.power_to_db(spec), origin='lower', aspect=aspect)
  if xmax:
    axs.set_xlim((0, xmax))
  fig.colorbar(im, ax=axs)
  plt.show(block=False)

def get_spectrogram(
    n_fft = 400,
    win_len = None,
    hop_len = None,
    power = 2.0,
):
  waveform, _ = get_speech_sample()
  spectrogram = T.Spectrogram(
      n_fft=n_fft,
      win_length=win_len,
      hop_length=hop_len,
      center=True,
      pad_mode="reflect",
      power=power,
  )
  return spectrogram(waveform)
def resample_wav_files(input_path, output_path, target_sr):
    # Create the output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Get a list of all WAV files in the input directory and its subfolders
    for root, dirs, files in os.walk(input_path):
        for file_name in files:
            if file_name.endswith('.wav'):
                # Read the input WAV file
                input_file = os.path.join(root, file_name)
                audio, sr = librosa.load(input_file, sr=target_sr)

                # Write the resampled audio to the output WAV file
                output_dir = os.path.join(output_path, os.path.relpath(root, input_path))
                os.makedirs(output_dir, exist_ok=True)
                output_file = os.path.join(output_dir, file_name)
                sf.write(output_file, audio, target_sr)
def copy_wav_files_to_single_directory(input_path, output_path):
    # Create the output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Get a list of all WAV files in the input directory and its subfolders
    for root, dirs, files in os.walk(input_path):
        for file_name in files:
            if file_name.endswith('.wav'):
                # Copy the WAV file to the output directory with the original filename
                input_file = os.path.join(root, file_name)
                output_file = os.path.join(output_path, file_name)
                shutil.copy2(input_file, output_file)

### Spectral Subtraction method

#### Define the path to denoise 

In [3]:
bahna_dataset = sorted(list(Path(r"D:\Master\Thesis\FRCRN\Summary\Test").rglob('*.wav')))
bahna_dataset[0]

WindowsPath('D:/Master/Thesis/FRCRN/Summary/Test/0001.1.wav')

In [4]:
# dataframe to record statistics
stat_bahna_spectral_subtraction = pd.DataFrame(columns=['Name','Max', 'Min', 'Mean','Std','Noise_level_before_denoised','Max_after_denoised', 'Min_after_denoised', 'Mean_after_denoised','Std_after_denoised','Noise_level_after_denoised'])
stat_bahna_spectral_subtraction

,Name,Max,Min,Mean,Std,Noise_level_before_denoised,Max_after_denoised,Min_after_denoised,Mean_after_denoised,Std_after_denoised,Noise_level_after_denoised


In [6]:
for i in bahna_dataset:
    print(i)
    waveform, sample_rate = torchaudio.load(i)
    max_,min_,mean_,std_ = get_stats(waveform, sample_rate = sample_rate)
    if std_!=0 :
        noise_level = 10*( np.log10(std_/max_))
    else:
        noise_level = 0
    normalized_path = os.path.normpath(i)
    input_path = os.path.splitext(normalized_path)[0] + os.path.splitext(normalized_path)[1]
    denoised_path = os.path.splitext(normalized_path)[0] + "_spectral_subtraction_denoised" + os.path.splitext(normalized_path)[1]
    snd = parselmouth.Sound(input_path)
    snd_denoised = snd.copy()
    snd_denoised = call(snd_denoised, "Reduce noise", 0.0, 0.0, 0.025, 80.0, 10000.0, 40.0,noise_level, "Spectral-subtraction")
    # Save the denoised audio
    snd_denoised.save(denoised_path, "WAV")
    print(denoised_path)
    waveform_denoised, sample_rate_denoised = torchaudio.load(denoised_path)
    max_after,min_after,mean_after,std_after=get_stats(waveform_denoised, sample_rate=sample_rate_denoised)
    if std_after!=0 :
        noise_level_after = 10*( np.log10(std_after/max_after))
    else:
        noise_level_after = 0
    df = pd.DataFrame({"Name":[i],"Max":[max_],"Min":[min_],"Mean":[mean_],"Std":[std_],"Noise_level_before_denoised":[noise_level],'Max_after_denoised':[max_after], 'Min_after_denoised':[min_after], 'Mean_after_denoised':[mean_after],'Std_after_denoised':[std_after],'Noise_level_after_denoised':[noise_level_after]})
    stat_bahna_spectral_subtraction=pd.concat([stat_bahna_spectral_subtraction,df], ignore_index=True)
    

D:\Master\Thesis\FRCRN\Summary\Test\0001.1.wav
D:\Master\Thesis\FRCRN\Summary\Test\0001.1_spectral_subtraction_denoised.wav
D:\Master\Thesis\FRCRN\Summary\Test\0001.2.wav
D:\Master\Thesis\FRCRN\Summary\Test\0001.2_spectral_subtraction_denoised.wav
D:\Master\Thesis\FRCRN\Summary\Test\0001.3.wav
D:\Master\Thesis\FRCRN\Summary\Test\0001.3_spectral_subtraction_denoised.wav
D:\Master\Thesis\FRCRN\Summary\Test\0001.4.wav
D:\Master\Thesis\FRCRN\Summary\Test\0001.4_spectral_subtraction_denoised.wav
D:\Master\Thesis\FRCRN\Summary\Test\0001.5.wav
D:\Master\Thesis\FRCRN\Summary\Test\0001.5_spectral_subtraction_denoised.wav


In [7]:
stat_bahna_spectral_subtraction.head()

,Name,Max,Min,Mean,Std,Noise_level_before_denoised,Max_after_denoised,Min_after_denoised,Mean_after_denoised,Std_after_denoised,Noise_level_after_denoised
0,D:\Master\Thesis\FRCRN\Summary\Test\0001.1.wav,0.6552124,-0.7651062,-3.0624975e-05,0.07685376,-9.307170,0.6543884,-0.76400757,6.9447053e-09,0.07664716,-9.313395
1,D:\Master\Thesis\FRCRN\Summary\Test\0001.2.wav,0.55578613,-0.4998474,-3.9136925e-05,0.0951897,-7.663178,0.5445862,-0.47906494,8.370825e-07,0.09144629,-7.749005
2,D:\Master\Thesis\FRCRN\Summary\Test\0001.3.wav,0.47247314,-0.53967285,-2.9462188e-05,0.048951853,-9.846080,0.45462036,-0.49050903,1.8565483e-05,0.04017664,-10.536753
3,D:\Master\Thesis\FRCRN\Summary\Test\0001.4.wav,0.4461975,-0.56170654,-4.4831922e-05,0.059409264,-8.756730,0.4430542,-0.55825806,-3.5348178e-06,0.058748286,-8.774617
4,D:\Master\Thesis\FRCRN\Summary\Test\0001.5.wav,0.32443237,-0.37664795,-1.7288974e-05,0.046413366,-8.444811,0.32302856,-0.37149048,1.2664118e-06,0.04507507,-8.553045


!pip install openpyxl

### FRCRN method

This model use a fixed input with Sampling frequency is 16khz so we have to resample the input before we put it into our model

In [12]:
# Example usage resample to 16khz
input_directory = r'D:/Master/Thesis/FRCRN/Summary/Test/'
output_directory = 'D:/Master/Thesis/FRCRN/Summary/Test resample'
target_sampling_rate = 16000

resample_wav_files(input_directory, output_directory, target_sampling_rate)

#### Define path to denoised

In [20]:
bahna_dataset_FRCRN = sorted(list(Path(r"D:\Master\Thesis\FRCRN\Summary\Test resample").rglob('*.wav')))
bahna_dataset_FRCRN[0]

WindowsPath('D:/Master/Thesis/FRCRN/Summary/Test resample/0001.1.wav')

In [21]:
stat_bahna_FRCRN = pd.DataFrame(columns=['Name','Max', 'Min', 'Mean','Std','Noise_level_before_denoised','Max_after_denoised', 'Min_after_denoised', 'Mean_after_denoised','Std_after_denoised','Noise_level_after_denoised'])
stat_bahna_FRCRN

,Name,Max,Min,Mean,Std,Noise_level_before_denoised,Max_after_denoised,Min_after_denoised,Mean_after_denoised,Std_after_denoised,Noise_level_after_denoised


In [22]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks


In [23]:
for i in bahna_dataset_FRCRN:
    print(i)
    waveform, sample_rate = torchaudio.load(i)
    max_,min_,mean_,std_ = get_stats(waveform, sample_rate = sample_rate)
    if std_!=0 :
        noise_level = 10*( np.log10(std_/max_))
    else:
        noise_level = 0
    # df = pd.DataFrame({"Name":[i],"Std":[std_],"Noise_level":[noise_level]})
    # stat_bahna=pd.concat([stat_bahna,df], ignore_index=True)
    normalized_path = os.path.normpath(i)
    input_path = os.path.splitext(normalized_path)[0] + os.path.splitext(normalized_path)[1]
    denoised_path = os.path.splitext(normalized_path)[0] + "_denoised_FRCRN" + os.path.splitext(normalized_path)[1]
    
    # denoised audio    
    ans = pipeline(
        Tasks.acoustic_noise_suppression,
        model='damo/speech_frcrn_ans_cirm_16k')
    result = ans(
        str(i),
        output_path=denoised_path)
    print(denoised_path)
    waveform_denoised, sample_rate_denoised = torchaudio.load(denoised_path)
    max_after,min_after,mean_after,std_after=get_stats(waveform_denoised, sample_rate=sample_rate_denoised)
    if std_after!=0 :
        noise_level_after = 10*( np.log10(std_after/max_after))
    else:
        noise_level_after = 0
    df_FRCRN = pd.DataFrame({"Name":[i],"Max":[max_],"Min":[min_],"Mean":[mean_],"Std":[std_],"Noise_level_before_denoised":[noise_level],'Max_after_denoised':[max_after], 'Min_after_denoised':[min_after], 'Mean_after_denoised':[mean_after],'Std_after_denoised':[std_after],'Noise_level_after_denoised':[noise_level_after]})
    stat_bahna_FRCRN=pd.concat([stat_bahna_FRCRN,df_FRCRN], ignore_index=True)

D:\Master\Thesis\FRCRN\Summary\Test resample\0001.1.wav


2024-01-08 01:20:21,671 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.1
2024-01-08 01:20:22,606 - modelscope - INFO - initiate model from C:\Users\PC\.cache\modelscope\hub\damo\speech_frcrn_ans_cirm_16k
2024-01-08 01:20:22,606 - modelscope - INFO - initiate model from location C:\Users\PC\.cache\modelscope\hub\damo\speech_frcrn_ans_cirm_16k.
2024-01-08 01:20:22,611 - modelscope - INFO - initialize model from C:\Users\PC\.cache\modelscope\hub\damo\speech_frcrn_ans_cirm_16k
2024-01-08 01:20:23,298 - modelscope - WARNING - No preprocessor field found in cfg.
2024-01-08 01:20:23,298 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-01-08 01:20:23,306 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': 'C:\\Users\\PC\\.cache\\modelscope\\hub\\damo\\speech_frcrn_ans_cirm_16k'}. trying to build by task and model information.


inputs:(1, 104000)
padding: 20000
inputs after padding:(1, 124000)
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.1_denoised_FRCRN.wav
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.2.wav


2024-01-08 01:20:25,936 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.1
2024-01-08 01:20:26,466 - modelscope - INFO - initiate model from C:\Users\PC\.cache\modelscope\hub\damo\speech_frcrn_ans_cirm_16k
2024-01-08 01:20:26,466 - modelscope - INFO - initiate model from location C:\Users\PC\.cache\modelscope\hub\damo\speech_frcrn_ans_cirm_16k.
2024-01-08 01:20:26,470 - modelscope - INFO - initialize model from C:\Users\PC\.cache\modelscope\hub\damo\speech_frcrn_ans_cirm_16k
2024-01-08 01:20:27,316 - modelscope - WARNING - No preprocessor field found in cfg.
2024-01-08 01:20:27,322 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-01-08 01:20:27,323 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': 'C:\\Users\\PC\\.cache\\modelscope\\hub\\damo\\speech_frcrn_ans_cirm_16k'}. trying to build by task and model information.


inputs:(1, 55913)
padding: 19913
inputs after padding:(1, 75826)
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.2_denoised_FRCRN.wav
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.3.wav


...

inputs:(1, 27509)
padding: 491
inputs after padding:(1, 28000)
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.3_denoised_FRCRN.wav
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.4.wav


2024-01-08 01:20:33,485 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.1
2024-01-08 01:20:34,986 - modelscope - INFO - initiate model from C:\Users\PC\.cache\modelscope\hub\damo\speech_frcrn_ans_cirm_16k
2024-01-08 01:20:34,986 - modelscope - INFO - initiate model from location C:\Users\PC\.cache\modelscope\hub\damo\speech_frcrn_ans_cirm_16k.
2024-01-08 01:20:34,991 - modelscope - INFO - initialize model from C:\Users\PC\.cache\modelscope\hub\damo\speech_frcrn_ans_cirm_16k
2024-01-08 01:20:35,695 - modelscope - WARNING - No preprocessor field found in cfg.
2024-01-08 01:20:35,697 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-01-08 01:20:35,698 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': 'C:\\Users\\PC\\.cache\\modelscope\\hub\\damo\\speech_frcrn_ans_cirm_16k'}. trying to build by task and model information.


inputs:(1, 42707)
padding: 18707
inputs after padding:(1, 61414)
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.4_denoised_FRCRN.wav
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.5.wav


2024-01-08 01:20:37,844 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.1
2024-01-08 01:20:38,376 - modelscope - INFO - initiate model from C:\Users\PC\.cache\modelscope\hub\damo\speech_frcrn_ans_cirm_16k
2024-01-08 01:20:38,376 - modelscope - INFO - initiate model from location C:\Users\PC\.cache\modelscope\hub\damo\speech_frcrn_ans_cirm_16k.
2024-01-08 01:20:38,389 - modelscope - INFO - initialize model from C:\Users\PC\.cache\modelscope\hub\damo\speech_frcrn_ans_cirm_16k
2024-01-08 01:20:39,092 - modelscope - WARNING - No preprocessor field found in cfg.
2024-01-08 01:20:39,092 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-01-08 01:20:39,092 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': 'C:\\Users\\PC\\.cache\\modelscope\\hub\\damo\\speech_frcrn_ans_cirm_16k'}. trying to build by task and model information.


inputs:(1, 31086)
padding: 19086
inputs after padding:(1, 50172)
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.5_denoised_FRCRN.wav


In [24]:
stat_bahna_FRCRN.head()

,Name,Max,Min,Mean,Std,Noise_level_before_denoised,Max_after_denoised,Min_after_denoised,Mean_after_denoised,Std_after_denoised,Noise_level_after_denoised
0,D:\Master\Thesis\FRCRN\Summary\Test resample\0...,0.6415405,-0.744812,-4.554191e-05,0.07685028,-9.215786,0.22348022,-0.2592163,-1.2544192e-05,0.02678586,-9.213335
1,D:\Master\Thesis\FRCRN\Summary\Test resample\0...,0.5562134,-0.5000305,-5.428519e-05,0.095189065,-7.666544,0.14968872,-0.14984131,-6.419209e-06,0.025932873,-7.613384
2,D:\Master\Thesis\FRCRN\Summary\Test resample\0...,0.4666748,-0.539917,-4.4708602e-05,0.048952047,-9.792435,0.25463867,-0.28683472,-1.7360486e-05,0.024598626,-10.150135
3,D:\Master\Thesis\FRCRN\Summary\Test resample\0...,0.446167,-0.56173706,-6.0049035e-05,0.059409186,-8.756438,0.19506836,-0.25405884,-1.7696579e-05,0.024957675,-8.929827
4,D:\Master\Thesis\FRCRN\Summary\Test resample\0...,0.31622314,-0.37667847,-3.2520275e-05,0.046413545,-8.333489,0.17993164,-0.2157898,-9.342977e-06,0.02612493,-8.380524


### Evaluation (PESQ, STOI)

PESQ Score only support for sampling rate Fs = 8khz (defined as narrow band) or 16khz (defined as wide band) only so we have to resample it

In [54]:
# Example usage resample to 16khz
# input_directory = r'D:/Master/Thesis/FRCRN/Summary/Test/'
# output_directory = 'D:/Master/Thesis/FRCRN/Summary/Test resample'
# target_sampling_rate = 16000

# resample_wav_files(input_directory, output_directory, target_sampling_rate)

In [ ]:
# Example usage Datasets/clean_testset_wav
# input_directory = 'D:\Master\Thesis\FRCRN\CTV bana to add noise\Bình Định'
# output_file = 'D:\Master\Thesis\FRCRN\CTV bana to add noise\Bình Định only'

# copy_wav_files_to_single_directory(input_directory, output_file)

In [6]:
m = 'D:/Master/Thesis/FRCRN/Summary/Test resample/0001.1.wav'
n = 'D:/Master/Thesis/FRCRN/Summary/Test resample/0001.1_denoised.wav'
waveform, sample_rate = torchaudio.load(m)
waveform_denoised, sample_rate_denoised = torchaudio.load(n)

!pip install torchmetrics

In [31]:
from torchmetrics.audio import PerceptualEvaluationSpeechQuality
# pesq = PerceptualEvaluationSpeechQuality(8000, 'nb')
# pesq(preds, target)

wb_pesq = PerceptualEvaluationSpeechQuality(sample_rate, 'wb')
wb_pesq(waveform, waveform_denoised)

tensor(4.1132)

In [32]:
from torchmetrics.audio.stoi import ShortTimeObjectiveIntelligibility
stoi = ShortTimeObjectiveIntelligibility(sample_rate, False)
stoi(waveform, waveform_denoised)
#stoi(waveform_denoised, waveform)

tensor(0.9992)

In [49]:
metrics = pd.DataFrame(columns=['PESQ','STOI'])

In [50]:
for i in bahna_dataset_FRCRN:
    print(i)
    waveform, sample_rate = torchaudio.load(i)
    #normalized_path = os.path.normpath(i)
    denoised_path = os.path.splitext(i)[0] + "_denoised_FRCRN" + os.path.splitext(i)[1]
    waveform_denoised, sample_rate_denoised = torchaudio.load(denoised_path)
    print(denoised_path)
    wb_pesq = PerceptualEvaluationSpeechQuality(sample_rate, 'wb')
    stoi = ShortTimeObjectiveIntelligibility(sample_rate, False)
    metric = pd.DataFrame({"PESQ":[wb_pesq(waveform, waveform_denoised)],"STOI":[stoi(waveform, waveform_denoised)]})
    metrics = pd.concat([metrics,metric], ignore_index=True)

D:\Master\Thesis\FRCRN\Summary\Test resample\0001.1.wav
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.1_denoised_FRCRN.wav
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.2.wav
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.2_denoised_FRCRN.wav
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.3.wav
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.3_denoised_FRCRN.wav
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.4.wav
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.4_denoised_FRCRN.wav
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.5.wav
D:\Master\Thesis\FRCRN\Summary\Test resample\0001.5_denoised_FRCRN.wav


In [51]:
metrics

,PESQ,STOI
0,tensor(4.1132),tensor(0.9992)
1,tensor(1.9175),tensor(0.9248)
2,tensor(2.6582),tensor(0.9736)
3,tensor(2.6803),tensor(0.9624)
4,tensor(4.3992),tensor(0.9952)
